<a href="https://colab.research.google.com/github/KorStats/skku/blob/main/%EC%B5%9C%EC%A2%85code/(%EC%A7%84%EC%A7%9C_%EC%B5%9C%EC%A2%85)MLM_%EA%B3%A0%EB%8F%84%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers torch

In [ ]:
pip install konlpy

In [ ]:
from transformers import pipeline, AutoTokenizer
from konlpy.tag import Okt
import random

def mask_and_predict(sentence, model_name="beomi/KcBERT-base", num_masks=2, top_k=5):

    # 형태소 분석기(Okt) 사용하여 단어 분리
    okt = Okt()
    words = okt.morphs(sentence)  # 형태소 단위로 분리

    # BERT 모델 및 토크나이저 로드
    mlm_pipeline = pipeline("fill-mask", model=model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # 마스킹할 후보 단어 선택 (명사, 동사 우선)
    maskable_indices = [i for i, word in enumerate(words) if word not in ['을', '를', '이', '가', '은', '는', '에', '도', '의', '한', '가장']]

    # 마스킹할 단어 개수 설정
    num_masks = min(num_masks, len(maskable_indices))
    mask_indices = random.sample(maskable_indices, num_masks)

    # 마스킹 적용
    for idx in mask_indices:
        words[idx] = "[MASK]"

    # 마스킹된 문장 생성
    masked_sentence = " ".join(words)

    # MLM 모델로 예측 수행
    predictions = mlm_pipeline(masked_sentence, top_k=top_k)

    # 여러 개의 [MASK]를 예측하여 문장 복원
    new_sentence = masked_sentence
    for i, mask_preds in enumerate(predictions):
        best_pred = random.choice([pred["token_str"] for pred in mask_preds[:3]])  # Top 3 중 랜덤 선택
        new_sentence = new_sentence.replace("[MASK]", best_pred, 1)

    return new_sentence


# 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
'Colab Notebooks'  'My Drive'   NSMC_0.02.pkl   NSMC_0.5.pkl   이전   졸업논문


In [ ]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/졸업논문/DATA/NSMC_0.5_증분대상_2500개.csv', encoding='utf-8-sig')

In [ ]:
df['document'].isnull().sum()

0

In [ ]:
df['MLM_aug'] = df['document'].apply(mask_and_predict)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Some weights of the model checkpoint at beomi/KcBERT-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
Some weights of the model checkpoint at beomi/KcBERT-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are ini

In [ ]:
df.head(5)

,id,document,label,길이,MLM_aug
0,7652286,솔직히 이영화때문에 슈퍼맨시리즈가 7년이란 세월동안 묻혀있엇다 맨오브스틸은 미국에서...,0,138,솔직히 이영화 때문 에 슈퍼맨 시리즈 가 7년 이란 세월 동안 묻혀있엇다 맨오브스틸...
1,8895340,아무리 짱짱맨 옵대장의 다때려부수는 CG가 좋다그래도 이딴 디워랑 견줄만한 스토리로...,0,101,아무리 짱짱맨 옵 대장 의 다 때려 부수 는 CG 가 좋다 그래도 이딴 영화 랑 견...
2,6549317,오프닝장면 죠스영화 고대루 베겼네?? 여자 비명 연기하는 것도 고대루네~! 그냥 갖...,0,103,오프닝 장면 죠스 영화 고 대루 베 ##팅 ?? 여자 비명 연기 하는 것 도 고 대...
3,8650550,대표적인 친일선전영화라서 최하점을 줄수밖에 없다. 영화수준 자체도 초딩 학예회수준의...,0,116,대표 적 인 중국 선전 영화 라서 최하 점 을 줄수밖에 없다 . 영화 그 자체 도 ...
4,8658070,아~ 그렇구나~ 남편이 두 여자랑 바람펴도 조강지처 잘못인거구나~ㅎㅎ 부재는 '회개...,0,137,아 ~ 그렇구나 ~ 남편 이 두 여자 랑 바람 펴도 조강지처 잘못 인거 구나 ~ ㅎ...


In [ ]:
df.to_csv('NSMC_0.5_aug_mlm_kcbert_고도화.csv', encoding='utf-8-sig')